In [1]:
import time
from glob import glob
import os, argparse
import tensorflow as tf
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt

import helper
import labels

In [2]:
# Make sure you've already frozen the graph of the KITTI model
def load_graph(frozen_graph_filename='checks_KITTI/frozen_model.pb'):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [3]:
#You will need to change the data folder to direct to the KITTI training set
data_folder = '/Volumes/Seagate/Code/udacity_SDC/term3/CarND-Semantic-Segmentation/data/training'
orig_shape = (375, 1242)
img_shape = (256, 856)
mask_shape = img_shape
get_batches_fn = helper.gen_batch_function_KITTI(data_folder=data_folder, image_shape=img_shape, 
                                                 mask_shape=img_shape, num_classes=2, mode='test')
batch_size = 10

tf.reset_default_graph()
graph = load_graph(frozen_graph_filename='checks_KITTI/frozen_model.pb')

labels = []
predictions = []

with tf.Session(graph=graph) as sess:
    x = graph.get_tensor_by_name("prefix/x:0")
    train_mode = graph.get_tensor_by_name("prefix/train_mode:0")
    preds = graph.get_tensor_by_name("prefix/preds:0")
    
    #cnt = 0
    all_preds = None
    all_x = None
    all_y = None
    for batch_x, batch_y, _ in get_batches_fn(batch_size, mode='val'):
        feed_dict ={x: batch_x,
                train_mode: False}
        batch_preds = sess.run(preds,feed_dict)
        
        if all_preds is not None:
            all_preds = np.concatenate((all_preds, batch_preds), axis=0)
            all_x = np.concatenate((all_x, batch_x), axis=0)
            all_y = np.concatenate((all_y, batch_y), axis=0)

        else:
            all_preds = batch_preds
            all_x = batch_x
            all_y = batch_y
        
        #cnt += 1
        #if cnt > 1: 
           # break

In [4]:
correct_road = 0
correct_void = 0
for i in range(len(all_preds)):
    pred = np.argmax(all_preds[i], axis=-1)
    label = np.argmax(all_y[i], axis=-1)
    combined = pred + label
    values, cnts = np.unique(combined, return_counts=True)
    n_cor_road = 0
    n_cor_void = 0
    for j in range(len(values)):
        if values[j] == 0:
            n_cor_road = cnts[j]
        if values[j] == 2:
            n_cor_void = cnts[j]
    #[n_cor_road, n_wrong, n_cor_void]
    per_road = n_cor_road / len(label[label==0])
    per_void = n_cor_void / len(label[label==1])
    correct_road += per_road
    correct_void += per_void

correct_road /= len(all_preds)
correct_void /= len(all_preds)

print('Percent of road labelled correctly: {:.2f}%'.format(correct_road*100))
print('Percent of background labelled correctly: {:.2f}%'.format(correct_void*100))

Percent of road labelled correctly: 98.26%
Percent of background labelled correctly: 98.82%
